In [1]:
import json
movie_list = []
with open("movie_dict_3.json", "r") as fin:
    for i, line in enumerate(fin):
        plot_dict = {}
        movie_json = json.loads(line)
        plot_dict['imdb_id'] = movie_json['imdb_id']
        plot_dict['summary'] = movie_json['Plot']
        movie_list.append(plot_dict)

In [2]:
len(movie_list)

14820

In [3]:
import re
import nltk
# nltk.download('stopwords')
from nltk import word_tokenize,sent_tokenize
import heapq

output_movie_list = []
for entry in movie_list:
    filename = "/Users/maitrayedas/Documents/Fall 2018/ML/project/movie/MPST/final_plots_wiki_imdb_combined/raw/" + entry['imdb_id'] + ".txt"
    with open(filename, "r") as fin:
        synopsis = fin.read()
    
    # Removing special characters and digits
    formatted_synopsis = re.sub('[^a-zA-Z]', ' ', synopsis )  
    formatted_synopsis = re.sub(r'\s+', ' ', formatted_synopsis)
    
    #tokenize sentence i.e., find sentences from the synopsis
    sentence_list = nltk.sent_tokenize(synopsis) 
    
    #remove stopwords and find word frequencies
    stopwords = nltk.corpus.stopwords.words('english')
    word_frequencies = {}  
    for word in nltk.word_tokenize(formatted_synopsis):  
        if word not in stopwords:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1
    
    #find weighted frequency
    maximum_frequncy = max(word_frequencies.values())
    for word in word_frequencies.keys():  
        word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)
       
    #calculating sentence frequency
    sentence_scores = {}  
    for sent in sentence_list:  
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies.keys():
                if len(sent.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word]
                    else:
                        sentence_scores[sent] += word_frequencies[word]
  
    #retrieve top 7 sentences with the highest scores
    summary_sentences = heapq.nlargest(5, sentence_scores, key=sentence_scores.get)
    summary = ' '.join(summary_sentences)
    entry['out_summary'] = summary
    output_movie_list.append(entry) 

In [4]:
len(output_movie_list)

14820

In [10]:
import rouge

def prepare_results(p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)

for aggregator in ['Avg', 'Best', 'Individual']:
    print('Evaluation with {}'.format(aggregator))
    apply_avg = aggregator == 'Avg'
    apply_best = aggregator == 'Best'

    print(apply_avg)
    print(apply_best)
    evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                           max_n=4,
                           limit_length=False,
                           length_limit=100,
                           length_limit_type='words',
                           apply_avg=apply_avg,
                           apply_best=apply_best,
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)
    
    all_hypothesis = []
    all_references = []
    for entry in output_movie_list:
        all_hypothesis.append(entry['out_summary'])
        all_references.append(entry['summary'])

    scores = evaluator.get_scores(all_hypothesis, all_references)

    for metric, results in sorted(scores.items(), key=lambda x: x[0]):
        if not apply_avg and not apply_best: # value is a type of list as we evaluate each summary vs each reference
            for hypothesis_id, results_per_ref in enumerate(results):
                nb_references = len(results_per_ref['p'])
                for reference_id in range(nb_references):
                    print('\tHypothesis #{} & Reference #{}: '.format(hypothesis_id, reference_id))
                    print('\t' + prepare_results(results_per_ref['p'][reference_id], results_per_ref['r'][reference_id], results_per_ref['f'][reference_id]))
            print()
        else:
            print(prepare_results(results['p'], results['r'], results['f']))
    print()

Evaluation with Avg
True
False
	rouge-1:	P: 36.31	R: 30.80	F1: 31.31
	rouge-2:	P:  9.90	R:  4.70	F1:  6.09
	rouge-3:	P:  7.61	R:  2.86	F1:  4.14
	rouge-4:	P:  7.08	R:  2.58	F1:  3.79
	rouge-l:	P: 25.63	R: 21.53	F1: 22.29
	rouge-w:	P: 13.96	R:  3.98	F1:  5.74

Evaluation with Best
False
True
	rouge-1:	P: 36.31	R: 30.80	F1: 31.31
	rouge-2:	P:  9.90	R:  4.70	F1:  6.09
	rouge-3:	P:  7.61	R:  2.86	F1:  4.14
	rouge-4:	P:  7.08	R:  2.58	F1:  3.79
	rouge-l:	P: 25.63	R: 21.53	F1: 22.29
	rouge-w:	P: 13.96	R:  3.98	F1:  5.74

Evaluation with Individual
False
False
	Hypothesis #0 & Reference #0: 
		rouge-1:	P: 11.94	R: 21.62	F1: 15.38
	Hypothesis #1 & Reference #0: 
		rouge-1:	P: 91.06	R: 33.73	F1: 49.23
	Hypothesis #2 & Reference #0: 
		rouge-1:	P: 49.06	R: 28.11	F1: 35.74
	Hypothesis #3 & Reference #0: 
		rouge-1:	P: 27.27	R: 40.74	F1: 32.67
	Hypothesis #4 & Reference #0: 
		rouge-1:	P: 28.57	R: 29.01	F1: 28.79
	Hypothesis #5 & Reference #0: 
		rouge-1:	P: 38.06	R: 26.15	F1: 31.00
	Hypothesis #6

In [5]:
import rouge

def prepare_results(p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)

evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                           max_n=4,
                           limit_length=False,
                           length_limit=100,
                           length_limit_type='words',
                           apply_avg=True,
                           apply_best=False,
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)

all_hypothesis = []
all_references = []
for entry in output_movie_list:
    all_hypothesis.append(entry['out_summary'])
    all_references.append(entry['summary'])

scores = evaluator.get_scores(all_hypothesis, all_references)

for metric, results in sorted(scores.items(), key=lambda x: x[0]):
    print(prepare_results(results['p'], results['r'], results['f']))

	rouge-1:	P: 25.32	R: 34.90	F1: 26.61
	rouge-2:	P:  3.97	R:  4.90	F1:  3.93
	rouge-3:	P:  1.24	R:  1.30	F1:  1.10
	rouge-4:	P:  0.75	R:  0.68	F1:  0.61
	rouge-l:	P: 18.40	R: 25.69	F1: 19.80
	rouge-w:	P:  9.04	R:  6.49	F1:  6.25


In [18]:
textteaser_movie_list = []
with open("output_textteaser.json", "r")as fin:
    for line in fin:
        movie_json = json.loads(line)
        textteaser_movie_list.append(movie_json)

In [19]:
print(len(textteaser_movie_list))

14820


In [20]:
textteaser_movie_list[2000]

{'imdb_id': 'tt1441395',
 'out_summary': 'She seems confused and scared, and in the struggle he not only rips her clothes but also her skin, to find she has another black skin underneath. The man from the club stares in shock as the empty skin of the first victim gruesomely floats away. The empty skin floats aimlessly in the black void, twisting like a plastic bag in the wind. Suddenly a sharp snap noise punctuates the silence, and the first victim POPS like a balloon- his skin stretching out and then shrivelling up, just like a popped balloon would. Something she has done has caused the motorcycle man to be displeased, causing him to somehow question or lecture her actions.',
 'summary': 'An alien entity inhabits the earthly form of a young woman who combs the roads and streets of Scotland in search of the human prey she came to plunder. She seduces her isolated and forsaken male victims into an otherworldly dimension where they are stripped and consumed. However, existence in all its

In [21]:
import rouge

def prepare_results(p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)

evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                           max_n=4,
                           limit_length=False,
                           length_limit=100,
                           length_limit_type='words',
                           apply_avg=True,
                           apply_best=False,
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)

all_hypothesis = []
all_references = []
for entry in textteaser_movie_list:
    all_hypothesis.append(entry['out_summary'])
    all_references.append(entry['summary'])

scores = evaluator.get_scores(all_hypothesis, all_references)

for metric, results in sorted(scores.items(), key=lambda x: x[0]):
    print(prepare_results(results['p'], results['r'], results['f']))

	rouge-1:	P: 26.51	R: 33.66	F1: 26.71
	rouge-2:	P:  4.41	R:  5.17	F1:  4.23
	rouge-3:	P:  1.43	R:  1.52	F1:  1.27
	rouge-4:	P:  0.84	R:  0.80	F1:  0.69
	rouge-l:	P: 19.45	R: 25.25	F1: 20.20
	rouge-w:	P:  9.78	R:  6.44	F1:  6.39
